<a href="https://colab.research.google.com/github/darmstater/Project/blob/main/Preprocessing_LastProjek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IMPORT LIBRARY

In [ ]:
import numpy as np
import pandas as pd
# import nltk
import re
# from nltk.stem import PorterStemmer
# from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
# from nltk.stem import WordNetLemmatizer
import itertools
from wordcloud import WordCloud
# from sklearn.ensemble import RandomForestClassifier
# from sklearn import tree
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC
# from sklearn.ensemble import BaggingClassifier
# from sklearn.tree import DecisionTreeClassifier
from google.colab import drive
# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# from sklearn.model_selection import GridSearchCV

## IMPORT DATASET

In [ ]:
drive.mount('/content/drive/') #

raw_data = pd.read_csv('/content/drive/MyDrive/SKRIPSI/diabetic_data.csv')
raw_data.head()

Mounted at /content/drive/


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [ ]:
raw_data.shape

(101766, 50)

## PRE-PROCESSING

### **FEATURE SELECTION**


Menghapus beberapa fitur/kolom yang {menurut artikel yang disertakan saat saya mengambil dataset, ada beberapa Nilai diagnosis utama dalam set data akhir. Dalam analisis, kelompok yang mencakup kurang dari 3,5% pertemuan dikelompokkan
ke dalam kategori "lainnya"}

In [ ]:
secondary_variable = ['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id', 'payer_code', 'medical_specialty']
raw_data.drop(columns=secondary_variable, axis=1, inplace=True)

raw_data.to_csv('/content/drive/MyDrive/SKRIPSI/cleaned_data.csv', index=False)

In [ ]:
cleaned_data = pd.read_csv('/content/drive/MyDrive/SKRIPSI/df_cleaned.csv')
cleaned_data.head(100)

# cleaned_data => data yang sudah dihapus kolom yang tidak terpakai dalam penelitian

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,1,41,0,1,0,0,0,250.83,?,?,...,No,No,No,No,No,No,No,No,No,NO
1,3,59,0,18,0,0,0,276,250.01,255,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,2,11,5,13,2,0,1,648,250,V27,...,No,No,No,No,No,No,No,No,Yes,NO
3,2,44,1,16,0,0,0,8,250.43,403,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,1,51,0,8,0,0,0,197,157,250,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,19,2,8,0,0,0,996,413,414,...,No,Steady,No,No,No,No,No,No,Yes,>30
96,5,64,0,21,0,0,1,38,428,427,...,No,Steady,No,No,No,No,No,No,Yes,>30
97,3,55,4,23,0,0,1,414,411,511,...,No,Down,No,No,No,No,No,Ch,Yes,NO
98,2,53,0,6,0,0,0,590,250.01,401,...,No,Up,No,No,No,No,No,Ch,Yes,NO


In [ ]:
cleaned_data.shape

(101766, 39)

###**DATA CLEANING**

bersihin data yang ? ato ngaco

In [ ]:
# Periksa dan ganti "?" dengan 0
cleaned_data = cleaned_data.replace("?", 0)
cleaned_data.head(10)

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,1,41,0,1,0,0,0,250.83,0,0,...,No,No,No,No,No,No,No,No,No,NO
1,3,59,0,18,0,0,0,276,250.01,255,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,2,11,5,13,2,0,1,648,250,V27,...,No,No,No,No,No,No,No,No,Yes,NO
3,2,44,1,16,0,0,0,8,250.43,403,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,1,51,0,8,0,0,0,197,157,250,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
5,3,31,6,16,0,0,0,414,411,250,...,No,Steady,No,No,No,No,No,No,Yes,>30
6,4,70,1,21,0,0,0,414,411,V45,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
7,5,73,0,12,0,0,0,428,492,250,...,No,No,No,No,No,No,No,No,Yes,>30
8,13,68,2,28,0,0,0,398,427,38,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
9,12,33,3,18,0,0,0,434,198,486,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


###**LABELING**


####**Data Segregation**

```
# This is formatted as code
```


Misahin Data INT dan Object

In [ ]:
cleaned_data.info() #untuk cek tipe data per kolom

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 39 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   time_in_hospital          101766 non-null  int64 
 1   num_lab_procedures        101766 non-null  int64 
 2   num_procedures            101766 non-null  int64 
 3   num_medications           101766 non-null  int64 
 4   number_outpatient         101766 non-null  int64 
 5   number_emergency          101766 non-null  int64 
 6   number_inpatient          101766 non-null  int64 
 7   diag_1                    101766 non-null  object
 8   diag_2                    101766 non-null  object
 9   diag_3                    101766 non-null  object
 10  number_diagnoses          101766 non-null  int64 
 11  max_glu_serum             101766 non-null  object
 12  A1Cresult                 101766 non-null  object
 13  metformin                 101766 non-null  object
 14  repa

In [ ]:
# Misahin Data Int dan Obj

int_columns = cleaned_data.select_dtypes(include="int64")
object_columns = cleaned_data.select_dtypes(include="object")

In [ ]:
# cek yang dipindah udh bener apa belom

int_columns.head(10)

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
0,1,41,0,1,0,0,0,1
1,3,59,0,18,0,0,0,9
2,2,11,5,13,2,0,1,6
3,2,44,1,16,0,0,0,7
4,1,51,0,8,0,0,0,5
5,3,31,6,16,0,0,0,9
6,4,70,1,21,0,0,0,7
7,5,73,0,12,0,0,0,8
8,13,68,2,28,0,0,0,8
9,12,33,3,18,0,0,0,8


In [ ]:
object_columns.head(10)

,diag_1,diag_2,diag_3,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,250.83,0,0,None,None,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,NO
1,276,250.01,255,None,None,No,No,No,No,No,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,648,250,V27,None,None,No,No,No,No,No,...,No,No,No,No,No,No,No,No,Yes,NO
3,8,250.43,403,None,None,No,No,No,No,No,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,197,157,250,None,None,No,No,No,No,No,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
5,414,411,250,None,None,No,No,No,No,No,...,No,Steady,No,No,No,No,No,No,Yes,>30
6,414,411,V45,None,None,Steady,No,No,No,Steady,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
7,428,492,250,None,None,No,No,No,No,No,...,No,No,No,No,No,No,No,No,Yes,>30
8,398,427,38,None,None,No,No,No,No,No,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
9,434,198,486,None,None,No,No,No,No,No,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


####**Label Encoding**

Cari Unique Value Per Kolom (untuk labeling)

In [ ]:
# Kolom-kolom yang Mau Dicek
cek_Uvalue = ['diag_1', 'diag_2', 'diag_3', 'max_glu_serum', 'A1Cresult', 'metformin',
               'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide',
               'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone',
               'acarbose', 'miglitol', 'troglitazone', 'tolazamide',
               'insulin', 'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone',
               'metformin-rosiglitazone', 'metformin-pioglitazone', 'change', 'diabetesMed',
               'readmitted']

# Dapatkan Unique Value untuk setiap kolom
for i in cek_Uvalue:
    unique_value = object_columns[i].unique()
    print(f"Unique Value dalam kolom {i}: {unique_value}")


Unique Value dalam kolom diag_1: ['250.83' '276' '648' '8' '197' '414' '428' '398' '434' '250.7' '157'
 '518' '999' '410' '682' '402' '737' '572' 'V57' '189' '786' '427' '996'
 '277' '584' '462' '473' '411' '174' '486' '998' '511' '432' '626' '295'
 '196' '250.6' '618' '182' '845' '423' '808' '250.4' '722' '403' '250.11'
 '784' '707' '440' '151' '715' '997' '198' '564' '812' '38' '590' '556'
 '578' '250.32' '433' 'V58' '569' '185' '536' '255' '250.13' '599' '558'
 '574' '491' '560' '244' '250.03' '577' '730' '188' '824' '250.8' '332'
 '562' '291' '296' '510' '401' '263' '438' '70' '250.02' '493' '642' '625'
 '571' '738' '593' '250.42' '807' '456' '446' '575' '250.41' '820' '515'
 '780' '250.22' '995' '235' '250.82' '721' '787' '162' '724' '282' '514'
 'V55' '281' '250.33' '530' '466' '435' '250.12' 'V53' '789' '566' '822'
 '191' '557' '733' '455' '711' '482' '202' '280' '553' '225' '154' '441'
 '250.81' '349' 0 '962' '592' '507' '386' '156' '200' '728' '348' '459'
 '426' '388' '607' '3

In [ ]:
# kolom examide dan citoglipton dihapus karena hanya mengandung 1 buah nilai yaitu "No"
hapus = ['examide', 'citoglipton']

# Menghapus kolom-kolom yang ada dalam list dari "hapus"
object_columns.drop(columns=hapus, axis=1, inplace=True)

object_columns.head(10)

,diag_1,diag_2,diag_3,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,...,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,250.83,0,0,None,None,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,NO
1,276,250.01,255,None,None,No,No,No,No,No,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,648,250,V27,None,None,No,No,No,No,No,...,No,No,No,No,No,No,No,No,Yes,NO
3,8,250.43,403,None,None,No,No,No,No,No,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,197,157,250,None,None,No,No,No,No,No,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
5,414,411,250,None,None,No,No,No,No,No,...,No,Steady,No,No,No,No,No,No,Yes,>30
6,414,411,V45,None,None,Steady,No,No,No,Steady,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
7,428,492,250,None,None,No,No,No,No,No,...,No,No,No,No,No,No,No,No,Yes,>30
8,398,427,38,None,None,No,No,No,No,No,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
9,434,198,486,None,None,No,No,No,No,No,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [ ]:
# Code ICD9
import re

# Fungsi untuk mengganti nilai dalam kolom berdasarkan karakter E dan V
def ganti_nilai(value):
    if pd.notna(value) and isinstance(value, str):
        if re.match(r'^E\d', value):
            return 10000
        elif re.match(r'^V\d', value):
            return 20000
    return value  # Biarkan nilai lainnya tetap sama

# Mengganti nilai dalam kolom diag_1, diag_2, dan diag_3
for col in ['diag_1', 'diag_2', 'diag_3']:
    object_columns[col] = object_columns[col].apply(ganti_nilai)

changed = object_columns[['diag_1', 'diag_2','diag_3']]

print(changed)

        diag_1  diag_2 diag_3
0       250.83       0      0
1          276  250.01    255
2          648     250  20000
3            8  250.43    403
4          197     157    250
...        ...     ...    ...
101761  250.13     291    458
101762     560     276    787
101763      38     590    296
101764     996     285    998
101765     530     530    787

[101766 rows x 3 columns]


In [ ]:
# cek tipe datane
object_columns .info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 29 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   diag_1                    101766 non-null  object
 1   diag_2                    101766 non-null  object
 2   diag_3                    101766 non-null  object
 3   max_glu_serum             101766 non-null  object
 4   A1Cresult                 101766 non-null  object
 5   metformin                 101766 non-null  object
 6   repaglinide               101766 non-null  object
 7   nateglinide               101766 non-null  object
 8   chlorpropamide            101766 non-null  object
 9   glimepiride               101766 non-null  object
 10  acetohexamide             101766 non-null  object
 11  glipizide                 101766 non-null  object
 12  glyburide                 101766 non-null  object
 13  tolbutamide               101766 non-null  object
 14  piog

In [ ]:
# Mengubah tipe data kolom diag_1, diag_2, dan diag_3 menjadi tipe data float dengan mengatasi nilai yang tidak valid
object_columns['diag_1'] = pd.to_numeric(object_columns['diag_1'], errors='coerce')
object_columns['diag_2'] = pd.to_numeric(object_columns['diag_2'], errors='coerce')
object_columns['diag_3'] = pd.to_numeric(object_columns['diag_3'], errors='coerce')

# Mengganti nilai NaN dengan nilai 0 (atau nilai lain yang sesuai)
object_columns['diag_1'].fillna(0, inplace=True)
object_columns['diag_2'].fillna(0, inplace=True)
object_columns['diag_3'].fillna(0, inplace=True)

# Mengubah tipe data menjadi int
object_columns['diag_1'] = object_columns['diag_1'].astype(int)
object_columns['diag_2'] = object_columns['diag_2'].astype(int)
object_columns['diag_3'] = object_columns['diag_3'].astype(int)

# cek tipe datane
object_columns .info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 29 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   diag_1                    101766 non-null  int64 
 1   diag_2                    101766 non-null  int64 
 2   diag_3                    101766 non-null  int64 
 3   max_glu_serum             101766 non-null  object
 4   A1Cresult                 101766 non-null  object
 5   metformin                 101766 non-null  object
 6   repaglinide               101766 non-null  object
 7   nateglinide               101766 non-null  object
 8   chlorpropamide            101766 non-null  object
 9   glimepiride               101766 non-null  object
 10  acetohexamide             101766 non-null  object
 11  glipizide                 101766 non-null  object
 12  glyburide                 101766 non-null  object
 13  tolbutamide               101766 non-null  object
 14  piog

In [ ]:
#diag_1
def label_diag(a):
    if 390 <= a <= 459 or a == 785:
        return 0
    elif 460 <= a <= 519 or a == 786:
        return 1
    elif 520 <= a <= 579 or a == 787:
        return 2
    elif a == 250:
        return 3
    elif 800 <= a <= 999:
        return 4
    elif 710 <= a <= 739:
        return 5
    elif 580 <= a <= 629 or a == 788:
        return 6
    elif 140 <= a <= 239:
        return 7
    elif a in [780, 781, 784, 790, 791, 792, 793, 794, 795, 796, 797, 798, 799]:
        return 8
    elif 240 <= a <= 279:
        return 9
    elif 680 <= a <= 709 or a == 782:
        return 10
    elif 1 <= a <= 139:
        return 11
    elif 290 <= a <= 319:
        return 12
    elif 280 <= a <= 289:
        return 13
    elif 320 <= a <= 359:
        return 14
    elif 630 <= a <= 679:
        return 15
    elif 360 <= a <= 389:
        return 16
    elif 740 <= a <= 759:
        return 17
    elif a == 20000:
        return 18
    elif a == 10000:
        return 19
    else:
        return -1  # Kategori tidak ditemukan

a = object_columns['diag_1']

object_columns['diag_1'] = a.apply(label_diag)

print(object_columns)


        diag_1  diag_2  diag_3 max_glu_serum A1Cresult metformin repaglinide  \
0            3       0       0          None      None        No          No   
1            9     250     255          None      None        No          No   
2           15     250   20000          None      None        No          No   
3           11     250     403          None      None        No          No   
4            7     157     250          None      None        No          No   
...        ...     ...     ...           ...       ...       ...         ...   
101761       3     291     458          None        >8    Steady          No   
101762       2     276     787          None      None        No          No   
101763      11     590     296          None      None    Steady          No   
101764       4     285     998          None      None        No          No   
101765       2     530     787          None      None        No          No   

       nateglinide chlorpropamide glime

In [ ]:
#diag_2
def label_diag(a):
    if 390 <= a <= 459 or a == 785:
        return 0
    elif 460 <= a <= 519 or a == 786:
        return 1
    elif 520 <= a <= 579 or a == 787:
        return 2
    elif a == 250:
        return 3
    elif 800 <= a <= 999:
        return 4
    elif 710 <= a <= 739:
        return 5
    elif 580 <= a <= 629 or a == 788:
        return 6
    elif 140 <= a <= 239:
        return 7
    elif a in [780, 781, 784, 790, 791, 792, 793, 794, 795, 796, 797, 798, 799]:
        return 8
    elif 240 <= a <= 279:
        return 9
    elif 680 <= a <= 709 or a == 782:
        return 10
    elif 1 <= a <= 139:
        return 11
    elif 290 <= a <= 319:
        return 12
    elif 280 <= a <= 289:
        return 13
    elif 320 <= a <= 359:
        return 14
    elif 630 <= a <= 679:
        return 15
    elif 360 <= a <= 389:
        return 16
    elif 740 <= a <= 759:
        return 17
    elif a == 20000:
        return 18
    elif a == 10000:
        return 19
    else:
        return -1  # Kategori tidak ditemukan

a = object_columns['diag_2']

object_columns['diag_2'] = a.apply(label_diag)

print(object_columns)


        diag_1  diag_2  diag_3 max_glu_serum A1Cresult metformin repaglinide  \
0            3      -1       0          None      None        No          No   
1            9       3     255          None      None        No          No   
2           15       3   20000          None      None        No          No   
3           11       3     403          None      None        No          No   
4            7       7     250          None      None        No          No   
...        ...     ...     ...           ...       ...       ...         ...   
101761       3      12     458          None        >8    Steady          No   
101762       2       9     787          None      None        No          No   
101763      11       6     296          None      None    Steady          No   
101764       4      13     998          None      None        No          No   
101765       2       2     787          None      None        No          No   

       nateglinide chlorpropamide glime

In [ ]:
#diag_3
def label_diag(a):
    if 390 <= a <= 459 or a == 785:
        return 0
    elif 460 <= a <= 519 or a == 786:
        return 1
    elif 520 <= a <= 579 or a == 787:
        return 2
    elif a == 250:
        return 3
    elif 800 <= a <= 999:
        return 4
    elif 710 <= a <= 739:
        return 5
    elif 580 <= a <= 629 or a == 788:
        return 6
    elif 140 <= a <= 239:
        return 7
    elif a in [780, 781, 784, 790, 791, 792, 793, 794, 795, 796, 797, 798, 799]:
        return 8
    elif 240 <= a <= 279:
        return 9
    elif 680 <= a <= 709 or a == 782:
        return 10
    elif 1 <= a <= 139:
        return 11
    elif 290 <= a <= 319:
        return 12
    elif 280 <= a <= 289:
        return 13
    elif 320 <= a <= 359:
        return 14
    elif 630 <= a <= 679:
        return 15
    elif 360 <= a <= 389:
        return 16
    elif 740 <= a <= 759:
        return 17
    elif a == 20000:
        return 18
    elif a == 10000:
        return 19
    else:
        return -1  # Kategori tidak ditemukan

a = object_columns['diag_3']

object_columns['diag_3'] = a.apply(label_diag)

print(object_columns)


        diag_1  diag_2  diag_3 max_glu_serum A1Cresult metformin repaglinide  \
0            3      -1      -1          None      None        No          No   
1            9       3       9          None      None        No          No   
2           15       3      18          None      None        No          No   
3           11       3       0          None      None        No          No   
4            7       7       3          None      None        No          No   
...        ...     ...     ...           ...       ...       ...         ...   
101761       3      12       0          None        >8    Steady          No   
101762       2       9       2          None      None        No          No   
101763      11       6      12          None      None    Steady          No   
101764       4      13       4          None      None        No          No   
101765       2       2       2          None      None        No          No   

       nateglinide chlorpropamide glime

In [ ]:
value_mapping = {
    "max_glu_serum": {"None": 0, ">300": 1, "Norm": 2, ">200": 3},
    "A1Cresult": {"None": 0, ">7": 1, ">8": 2, "Norm": 3},
    "metformin": {"No": 0, "Steady": 2, "Up": 1, "Down": 3},
    "repaglinide": {"No": 0, "Up": 1, "Steady": 2, "Down": 3},
    "nateglinide": {"No": 0, "Steady": 2, "Down": 3, "Up": 1},
    "chlorpropamide": {"No": 0, "Steady": 2, "Down": 3, "Up": 1},
    "glimepiride": {"No": 0, "Steady": 2, "Down": 3, "Up": 1},
    "acetohexamide": {"No": 0, "Steady": 1},
    "glipizide": {"No": 0, "Steady": 2, "Up": 1, "Down": 3},
    "glyburide": {"No": 0, "Steady": 2, "Up": 1, "Down": 3},
    "tolbutamide": {"No": 0, "Steady": 1},
    "pioglitazone": {"No": 0, "Steady": 2, "Up": 1, "Down": 3},
    "rosiglitazone": {"No": 0, "Steady": 2, "Up": 1, "Down": 3},
    "acarbose": {"No": 0, "Steady": 2, "Up": 1, "Down": 3},
    "miglitol": {"No": 0, "Steady": 2, "Down": 3, "Up": 1},
    "troglitazone": {"No": 0, "Steady": 1},
    "tolazamide": {"No": 0, "Steady": 2, "Up": 1},
    "insulin": {"No": 0, "Up": 1, "Steady": 2, "Down": 3},
    "glyburide-metformin": {"No": 0, "Steady": 2, "Down": 3, "Up": 1},
    "glipizide-metformin": {"No": 0, "Steady": 1},
    "glimepiride-pioglitazone": {"No": 0, "Steady": 1},
    "metformin-rosiglitazone": {"No": 0, "Steady": 1},
    "metformin-pioglitazone": {"No": 0, "Steady": 1},
    "change": {"No": 0, "Ch": 1},
    "diabetesMed": {"No": 0, "Yes": 1},
    "readmitted": {"NO": 0, ">30": 1, "<30": 2}
}

object_columns = object_columns.replace(value_mapping)

object_columns.head()

,diag_1,diag_2,diag_3,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,...,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,3,-1,-1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,3,9,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,1,1
2,15,3,18,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,11,3,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,1,0
4,7,7,3,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,1,1,0


In [ ]:
object_columns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 29 columns):
 #   Column                    Non-Null Count   Dtype
---  ------                    --------------   -----
 0   diag_1                    101766 non-null  int64
 1   diag_2                    101766 non-null  int64
 2   diag_3                    101766 non-null  int64
 3   max_glu_serum             101766 non-null  int64
 4   A1Cresult                 101766 non-null  int64
 5   metformin                 101766 non-null  int64
 6   repaglinide               101766 non-null  int64
 7   nateglinide               101766 non-null  int64
 8   chlorpropamide            101766 non-null  int64
 9   glimepiride               101766 non-null  int64
 10  acetohexamide             101766 non-null  int64
 11  glipizide                 101766 non-null  int64
 12  glyburide                 101766 non-null  int64
 13  tolbutamide               101766 non-null  int64
 14  pioglitazone        

###**Data Merge**

Menggabungkan data yang di pisah jadi 1

Dan menghapus 1 kolom target

In [ ]:
# Misalnya, object_columns berisi kolom objek dan int_columns berisi kolom integer
diabetesMed = 'diabetesMed'

# Gabungkan kembali kedua DataFrames
cleaned_data = pd.concat([object_columns, int_columns], axis=1)

cleaned_data.head(15)

,diag_1,diag_2,diag_3,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,...,diabetesMed,readmitted,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
0,3,-1,-1,0,0,0,0,0,0,0,...,0,0,1,41,0,1,0,0,0,1
1,9,3,9,0,0,0,0,0,0,0,...,1,1,3,59,0,18,0,0,0,9
2,15,3,18,0,0,0,0,0,0,0,...,1,0,2,11,5,13,2,0,1,6
3,11,3,0,0,0,0,0,0,0,0,...,1,0,2,44,1,16,0,0,0,7
4,7,7,3,0,0,0,0,0,0,0,...,1,0,1,51,0,8,0,0,0,5
5,0,0,3,0,0,0,0,0,0,0,...,1,1,3,31,6,16,0,0,0,9
6,0,0,18,0,0,2,0,0,0,2,...,1,0,4,70,1,21,0,0,0,7
7,0,1,3,0,0,0,0,0,0,0,...,1,1,5,73,0,12,0,0,0,8
8,0,0,11,0,0,0,0,0,0,0,...,1,0,13,68,2,28,0,0,0,8
9,0,7,1,0,0,0,0,0,0,0,...,1,0,12,33,3,18,0,0,0,8


In [ ]:
cleaned_data.to_csv('/content/drive/MyDrive/SKRIPSI/cleaned_data.csv', index=False)

#Data Subsampling

data_1000 = 1,000 data

data_2500 = 2,500 data

data_5000 = 5,000 data

data_10000 = 10,000 data

data_20000 = 20,000 data

data_50000 = 50,000 data

data_100000 = 100,000 data


**10000 Data**

In [ ]:
dataA = pd.read_csv('/content/drive/MyDrive/SKRIPSI/cleaned_data.csv', nrows=10000)
dataA.to_csv('/content/drive/MyDrive/SKRIPSI/dataA.csv', index=False)

dataA.shape



(10000, 37)

**25000 Data**

In [ ]:
dataB = pd.read_csv('/content/drive/MyDrive/SKRIPSI/cleaned_data.csv', nrows=25000)
dataB.to_csv('/content/drive/MyDrive/SKRIPSI/dataB.csv', index=False)

dataB.shape

(25000, 37)

**50000 Data**

In [ ]:
dataC = pd.read_csv('/content/drive/MyDrive/SKRIPSI/cleaned_data.csv', nrows=50000)
dataC.to_csv('/content/drive/MyDrive/SKRIPSI/dataC.csv', index=False)

dataC.shape

(50000, 37)

**5000 Data**

In [ ]:
dataD = pd.read_csv('/content/drive/MyDrive/SKRIPSI/cleaned_data.csv', nrows=10000)
dataD.to_csv('/content/drive/MyDrive/SKRIPSI/dataD.csv', index=False)

dataD.shape

(10000, 37)